## Load Libraries

In [1]:
import pandas as pd
# import modin.pandas as pd
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine
from urllib.parse import quote_plus

## Load Environment variables

In [2]:
# Load the .env file
load_dotenv()

# Fetch environment variables
db_address = os.getenv('DB_ADDRESS')
db_port = os.getenv('DB_PORT')
db_username = os.getenv('DB_USERNAME')
db_password = quote_plus(os.getenv('DB_PASSWORD'))
db_name = os.getenv('DB')

## Functions

In [3]:
def get_db_connection():
    """
    Establishes a connection to the PostgreSQL database using SQLAlchemy.
    - The `postgresql+psycopg2` dialect indicates that we're using PostgreSQL as the database backend.
    - The `{DB_USER}:{DB_PASS}` part specifies the username and password to use for connections.
    - The `@{DB_IP}:{DB_PORT}` part specifies the hostname and port number to use when connecting to the database.
    - The `/`${DB}` part specifies the database name or schema to connect to.

    :return: engine.connect(): An active connection object to the PostgreSQL database.
    """
    engine = create_engine(f'postgresql://{db_username}:{db_password}@{db_address}:{db_port}/{db_name}')

    # Establish a connection to the database:
    #   - This returns an active connection object, which can be used to execute queries and interact with the database.
    return engine.connect()

def fetch_data_from_sql(query):
    """
    Fetches data from a SQL table and returns it as a pandas DataFrame.

    :param query: (str) The SQL query to execute on the database.
    :return: pd.DataFrame: A pandas DataFrame containing the results of the SQL query.
    """

    # Establish a connection to the PostgreSQL database using get_db_connection():
    conn = get_db_connection()

    # Execute the SQL query on the database and store the result in a pandas DataFrame:
    df = pd.read_sql(query, conn)

    # Close the connection to the database to free up resources:
    conn.close()

    # Return the pandas DataFrame containing the results of the SQL query:
    return df

def change_column_names(column_list, std=True):
    if std:
        return [col.strip().title().replace('_', ' ') for col in column_list]

    else:
        return [col.strip().lower().replace(' ', '_') for col in column_list]

## Get Data

### Transaction

In [4]:
query_transaction = '''
    SELECT
        *
    FROM
        erp_data.funds_received.transactions
    '''

transaction = fetch_data_from_sql(query_transaction)

### Projects

In [5]:
 # Get existing master
query_project_dir_old = '''
        SELECT
            *
        FROM
            erp_data.funds_received.project_directory
'''

project_directory = fetch_data_from_sql(query_project_dir_old)

# Standardise column names
project_directory.columns = change_column_names(project_directory.columns)

### WBS Data

In [6]:
query_transaction = '''
        SELECT
            *
        FROM
            erp_data.funds_received.wbs
        '''

wbs = fetch_data_from_sql(query_transaction)

### Data Cleaning

In [7]:
# Remove extra spaces from column names
transaction_col = transaction.columns
transaction_col = [col.strip() for col in transaction_col]
transaction.columns = transaction_col

wbs_col = wbs.columns
wbs_col = [col.strip() for col in wbs_col]
wbs.columns = wbs_col

In [8]:
if transaction.shape[0] > 0 and wbs.shape[0] > 0 and project_directory.shape[0] > 0:
    # Perform WBS mapping
    transaction = transaction.merge(wbs[['WBS', 'WBS Details']].drop_duplicates(), how='left',
                                    left_on='Project definition', right_on='WBS')

    transaction = transaction.merge(wbs[['Sub WBS', 'SUB WBS Details']].drop_duplicates(), how='left',
                                    left_on='Object', right_on='Sub WBS')

    transaction['Project Id'] = transaction['SUB WBS Details'].apply(
        lambda x: str(x).strip().lower().replace(' ', '_'))

    # Assign Category
    transaction = transaction.merge(project_directory[['Project Id', 'Category']], how='left', on='Project Id').copy()

In [9]:
start_date = pd.to_datetime('01-04-2024', format='%d-%m-%Y')
end_date = pd.to_datetime('24-12-2024', format='%d-%m-%Y')

In [10]:
transaction['Ref. document number'] = transaction['Ref. document number'].fillna('').astype(str)

In [12]:
transaction['Document Number'] = transaction['Document Number'].fillna('').astype(str)

KeyError: 'Document Number'

### Cleaned Data

In [13]:
# Final Data
final_data = transaction[(transaction['Document Date'].between(start_date, end_date))]

In [14]:
final_data_bak = final_data.copy()

## Analysis

### Logic 1

#### Steps:
- For Indian Donation
    - Take all the DR data
    - Filter by Cost Element ≠ 550510, 550511
    - For each "Project Definition",
        - Get the total for each Project Definition
        - Get the corresponding "Ref. document number". There could be multiple document numbers.
        - For each document numbers,
            - Get all the SB entries
            - Get the total of all SB entries
        - Get the total of the SB entries and compare it with the total for each Project Definition
        - If the difference is same as total for each project definition, then use the original project definition as the Sub-WBS code
        - If the difference is less, then categorise the difference as 'Un-assigned' in the Sub-WBS code
    - Create a Dataframe of project wise breakup for each "Project Definition"


### Logic 2
- For each Projects in Transaction without any filter~ with only DR and SB entries
    - Pick the SB transaction for that project/wbs code
    - If there's no SB entry, use the DR entry directly
    - Perform sub-total and get rid of the records which nullify each other i.e. amount = 0
        - Of the remaining transactions,
            - Identify the reference document numbers
            - Prepare a dataset of these document numbers. All the entries will be SB
            - Exclude the original project/WBS code
    - Of the remaining transaction, get their corresponding ref. document numbers
        - For each document numbers,
            - Filter all corresponding SB transactions (filtered data with SB entries)
            - Exclude the original project/wbs code from the filtered dataset
            - If the value is less than 0, it's transferred to ()
            - If the value is greater than 0, it's transferred from

In [15]:
final_data = final_data_bak.copy()

In [72]:
import pandas as pd
# from itertools import combinations
from collections import defaultdict

# def find_nullifying_groups(df):
#     """
#     Identify groups of values in the 'Value in Obj. Crcy' column that nullify each other,
#     and return their corresponding 'Ref. document number'.
#
#     Args:
#     df (pd.DataFrame): The input DataFrame with columns 'Ref. document number' and 'Value in Obj. Crcy'.
#
#     Returns:
#     pd.DataFrame: A DataFrame containing the 'Ref. document number' and 'Value in Obj. Crcy'
#                   of nullifying groups.
#     """
#     # Initialize a set to store indices of rows that nullify each other
#     nullifying_indices = set()
#
#     # Extract 'Value in Obj. Crcy' and their indices
#     values = df['Value in Obj. Crcy'].tolist()
#     index_map = df.index.tolist()  # Map positional indices to actual DataFrame indices
#
#     n = len(values)
#
#     # Check all combinations of rows to find nullifying groups
#     for r in range(2, n + 1):  # At least 2 elements in the combination
#         for combo in combinations(range(n), r):
#             subset = [values[i] for i in combo]
#             if abs(sum(subset)) < 1e-9:  # Check for near-zero sum
#                 # Map positional indices to DataFrame indices
#                 nullifying_indices.update(index_map[i] for i in combo)
#
#     # Convert the set of indices to a list and filter the DataFrame
#     result = df.loc[list(nullifying_indices)]
#     return result

def find_nullifying_groups(df):
    """
    Efficiently find and return rows where values in 'Value in Obj. Crcy' nullify each other.

    Args:
    df (pd.DataFrame): The input DataFrame with columns 'Ref. document number' and 'Value in Obj. Crcy'.

    Returns:
    pd.DataFrame: A DataFrame containing the 'Ref. document number' and 'Value in Obj. Crcy'
                  of nullifying groups.
    """
    # Sort values by absolute amount in descending order (higher values first)
    df = df.reindex(df['Val/COArea Crcy'].abs().sort_values(ascending=False).index)

    # Dictionary to store unmatched values with their index positions
    unmatched = defaultdict(list)
    nullifying_indices = set()

    for idx, row in df.iterrows():
        value = row['Val/COArea Crcy']
        opposite_value = -value

        if opposite_value in unmatched:
            # Match with the first occurrence of opposite value
            match_idx = unmatched[opposite_value].pop(0)
            nullifying_indices.update([idx, match_idx])

            # Remove the key if no more unmatched values exist
            if not unmatched[opposite_value]:
                del unmatched[opposite_value]
        else:
            # Store index of the current unmatched value
            unmatched[value].append(idx)

    # Filter DataFrame to include only nullifying rows
    return df.loc[list(nullifying_indices)]


def get_fur(wbs_code):

    # Create an empty dataframe
    wbs_breakup = pd.DataFrame()

    # Check for SB entries
    df = transaction[
        (transaction['Project definition'] == wbs_code) &
        (transaction['Document type'] == 'SB')
    ]

    # Step 2
    if df.shape[0] == 0:
        wbs_breakup = pd.concat([
            wbs_breakup,
            transaction[
                transaction['Project definition'] == wbs_code
            ]
        ], ignore_index=True)

        return wbs_breakup

    else:
        # Sort
        df = df.sort_values(by=['Fiscal Year', 'Val/COArea Crcy'], ascending=[True, False]).copy()

        # Step 3
        df_nullified = find_nullifying_groups(df)

        # Step 4
        data_non_null = df[~df['Ref. document number'].isin(df_nullified['Ref. document number'])]

        # Step 5.1 and 5.2
        data_sb = transaction[transaction['Ref. document number'].isin(data_non_null['Ref. document number'])]

        # Step 5.3
        data_proj = data_sb[data_sb['Project definition'] != wbs_code]
        # data_proj = data_sb.copy()

        # Concatenate the final data
        project_breakup = pd.concat([wbs_breakup, data_proj], ignore_index=True)

        return project_breakup


### Logic 3

- Pick out only the DR Projects
- Ignore below project types
    - Legacy
    - HF Donations
    - CSR Donations
- Get the non-ignored data
- For the non-ignored project types, categorise it

In [50]:
final_data = final_data_bak.copy()

In [51]:
# Step 1
dr_data = final_data[final_data['Document type'] == 'DR'].copy()

In [52]:
dr_data['Val/COArea Crcy'].sum()

np.float64(-1535617446.1999998)

In [53]:
# Step 2.1
lp_data = dr_data[
    (dr_data['Project definition'].str.contains('LP')) |
    (dr_data['Object'].str.contains('LP'))
]

In [54]:
# Step 2.2
hf_data = dr_data[
    (dr_data['Project definition'].str.contains('HF')) |
    (dr_data['Object'].str.contains('HF'))
]

In [55]:
# Step 2.3
csr_data = dr_data[
    (dr_data['Project definition'].str.contains('CSRP')) |
    (dr_data['Object'].str.contains('CSRP'))
]

In [56]:
# Step 3
dr_data_non_ignored = dr_data[
    # Ignore Legacy Projects
    (
        ~(dr_data['Project definition'].isin(lp_data['Project definition'])) |
        ~(dr_data['Object'].isin(lp_data['Object']))
    ) &

    # Ignore HF Projects
    (
        ~(dr_data['Project definition'].isin(hf_data['Project definition'])) |
        ~(dr_data['Object'].isin(hf_data['Object']))
    ) &

    # Ignore CSR Projects
    (
        ~(dr_data['Project definition'].isin(csr_data['Project definition'])) |
        ~(dr_data['Object'].isin(csr_data['Object']))
    )
]

In [57]:
lp_data['Project definition'].drop_duplicates()

117724    DO/2000-LP74
118747    DO/2003-LP76
121480    DO/2014-CLLP
122403    DO/2017-LP92
124282    DO/2019-LP94
129739    DO/2021-LP96
132224    DO/2022-LP10
132228    DO/2022-LP97
134862    DO/2023-LP83
137626    DO/2024-LP08
137629    DO/2024-LP14
137630    DO/2024-LP99
Name: Project definition, dtype: object

In [73]:
for proj in lp_data['Project definition'].drop_duplicates().to_list():
    file_name = proj.replace('/', '_')
    get_fur(proj).to_csv(f'{file_name}.csv', index=False, quoting=1, lineterminator='\r\n')

In [64]:
# Project Breakup
fur_breakup = get_fur('DO/2000-LP74')

In [65]:
fur_breakup

,Ref. document number,Posting Date,Document Date,Document type,Project definition,Object,Fiscal Year,Cost Element,Cost element descr.,Offsetting acct no.,Name of offsetting account,Purchasing Document,Purchase order text,Val/COArea Crcy,WBS,WBS Details,Sub WBS,SUB WBS Details,Project Id,Category
0,2226006535,2022-11-02,2022-11-01,SB,DO/2003-HLRF,DO/2003-HLRFREV,2022,550508,Rev-Domestic (Indian fund),550508,Rev-Domestic,NaN,None,200000.0,DO/2003-HLRF,HOSTEL RENOVATION FUND,DO/2003-HLRFREV,HOSTEL RENOVATION FUND,hostel_renovation_fund,NaN
1,2226006535,2022-11-02,2022-11-01,SB,DO/2003-HLRF,DO/2003-HLRFREV,2022,550508,Rev-Domestic (Indian fund),550508,Rev-Domestic,NaN,None,5000.0,DO/2003-HLRF,HOSTEL RENOVATION FUND,DO/2003-HLRFREV,HOSTEL RENOVATION FUND,hostel_renovation_fund,NaN
2,2226009822,2023-01-16,2023-01-16,SB,DO/2005-BHFS,DO/2005-BHFS001,2022,550509,Rev-General Expence,550509,Rev-General Expence,NaN,None,8395372.6,DO/2005-BHFS,IIT BOMBAY HERITAGE FUND SCHOLARSHIP,DO/2005-BHFS001,IIT BOMBAY HERITAGE FUND SCHOLARSHIP,iit_bombay_heritage_fund_scholarship,NaN
3,2226007464,2022-11-22,2022-11-22,SB,DO/2006-RVNF,DO/2006-RVNF001,2022,550509,Rev-General Expence,550509,Rev-General Expence,NaN,None,4984111.0,DO/2006-RVNF,R.V.NILEKANI FUND FOR HOSTEL-8 MAINTENAN,DO/2006-RVNF001,R.V.NILEKANI FUND FOR HOSTEL-8 MAINTENAN,r.v.nilekani_fund_for_hostel-8_maintenan,NaN
4,2226007464,2022-11-22,2022-11-22,SB,DO/2006-RVNF,DO/2006-RVNFREV,2022,550508,Rev-Domestic (Indian fund),550509,Rev-General Expence,NaN,None,4000000.0,DO/2006-RVNF,R.V.NILEKANI FUND FOR HOSTEL-8 MAINTENAN,DO/2006-RVNFREV,R.V.NILEKANI FUND FOR HOSTEL-8 MAINTENAN,r.v.nilekani_fund_for_hostel-8_maintenan,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,2426010109,2024-12-27,2024-12-27,SB,DO/2022-ROAD,DO/2023-ROAD001-007,2024,550509,Rev-General Expence,550509,Rev-General Expence,NaN,None,2925912.6,DO/2022-ROAD,Research Projects on Pavements,DO/2023-ROAD001-007,Karyavattom Sports Facility Limited (KSF,karyavattom_sports_facility_limited_(ksf,NaN
125,2426010109,2024-12-27,2024-12-27,SB,DO/2022-ROAD,DO/2023-ROAD001-008,2024,550509,Rev-General Expence,550509,Rev-General Expence,NaN,None,60808811.0,DO/2022-ROAD,Research Projects on Pavements,DO/2023-ROAD001-008,Chenani Nashri Tunnelway Limited (CNTL),chenani_nashri_tunnelway_limited_(cntl),NaN
126,2426004221,2024-08-02,2024-08-02,SB,DO/2023-JANK,DO/2023-JANK002,2024,550508,Rev-Domestic (Indian fund),550508,Rev-Domestic,NaN,None,8313000.0,DO/2023-JANK,IITBHF GRANT 248,DO/2023-JANK002,Hostel Development,hostel_development,NaN
127,2426010109,2024-12-27,2024-12-27,SB,DO/2023-JANK,DO/2023-JANK002,2024,550509,Rev-General Expence,550509,Rev-General Expence,NaN,None,8319770.0,DO/2023-JANK,IITBHF GRANT 248,DO/2023-JANK002,Hostel Development,hostel_development,NaN


In [31]:
lp_data[lp_data['Project definition'] == 'DO/2021-LP96']

,Document Number,Ref. document number,Posting Date,Document Date,Document type,Project definition,Object,Fiscal Year,Cost Element,Cost element descr.,...,Vbl. value/TranCurr.,Vbl. value/Obj. curr,Var.val.in rep.cur.,Purchase order text,WBS,WBS Details,Sub WBS,SUB WBS Details,Project Id,Category
104650,1002064877.0,2406001261.0,2024-04-25,2024-04-23,DR,DO/2021-LP96,DO/2021-LP96REV,2024.0,550508.0,Rev-Domestic (Indian fund),...,-600000.0,-600000.0,-600000.0,None,DO/2021-LP96,Class of '96 Legacy Project,DO/2021-LP96REV,Class of '96 Legacy Project - REV,class_of_'96_legacy_project_-_rev,NaN
104651,1002127036.0,2406004839.0,2024-07-16,2024-07-09,DR,DO/2021-LP96,DO/2021-LP96REV,2024.0,550508.0,Rev-Domestic (Indian fund),...,-150000.0,-150000.0,-150000.0,None,DO/2021-LP96,Class of '96 Legacy Project,DO/2021-LP96REV,Class of '96 Legacy Project - REV,class_of_'96_legacy_project_-_rev,NaN
104652,1002133617.0,2406005018.0,2024-07-19,2024-07-15,DR,DO/2021-LP96,DO/2021-LP96REV,2024.0,550508.0,Rev-Domestic (Indian fund),...,-100000.0,-100000.0,-100000.0,None,DO/2021-LP96,Class of '96 Legacy Project,DO/2021-LP96REV,Class of '96 Legacy Project - REV,class_of_'96_legacy_project_-_rev,NaN
104653,1002133631.0,2406005028.0,2024-07-19,2024-07-15,DR,DO/2021-LP96,DO/2021-LP96REV,2024.0,550508.0,Rev-Domestic (Indian fund),...,-150000.0,-150000.0,-150000.0,None,DO/2021-LP96,Class of '96 Legacy Project,DO/2021-LP96REV,Class of '96 Legacy Project - REV,class_of_'96_legacy_project_-_rev,NaN
104654,1002133647.0,2406005033.0,2024-07-19,2024-07-16,DR,DO/2021-LP96,DO/2021-LP96REV,2024.0,550508.0,Rev-Domestic (Indian fund),...,-25001.0,-25001.0,-25001.0,None,DO/2021-LP96,Class of '96 Legacy Project,DO/2021-LP96REV,Class of '96 Legacy Project - REV,class_of_'96_legacy_project_-_rev,NaN
104655,1002136589.0,2406005555.0,2024-07-24,2024-07-17,DR,DO/2021-LP96,DO/2021-LP96REV,2024.0,550508.0,Rev-Domestic (Indian fund),...,-50000.0,-50000.0,-50000.0,None,DO/2021-LP96,Class of '96 Legacy Project,DO/2021-LP96REV,Class of '96 Legacy Project - REV,class_of_'96_legacy_project_-_rev,NaN
104656,1002161329.0,2406009215.0,2024-08-23,2024-08-19,DR,DO/2021-LP96,DO/2021-LP96REV,2024.0,550508.0,Rev-Domestic (Indian fund),...,-83888.0,-83888.0,-83888.0,None,DO/2021-LP96,Class of '96 Legacy Project,DO/2021-LP96REV,Class of '96 Legacy Project - REV,class_of_'96_legacy_project_-_rev,NaN
104657,1002164870.0,2406009731.0,2024-08-28,2024-08-24,DR,DO/2021-LP96,DO/2021-LP96REV,2024.0,550508.0,Rev-Domestic (Indian fund),...,-375000.0,-375000.0,-375000.0,None,DO/2021-LP96,Class of '96 Legacy Project,DO/2021-LP96REV,Class of '96 Legacy Project - REV,class_of_'96_legacy_project_-_rev,NaN
104659,1002181495.0,2406010278.0,2024-09-20,2024-09-13,DR,DO/2021-LP96,DO/2021-LP96REV,2024.0,550508.0,Rev-Domestic (Indian fund),...,-10000.0,-10000.0,-10000.0,None,DO/2021-LP96,Class of '96 Legacy Project,DO/2021-LP96REV,Class of '96 Legacy Project - REV,class_of_'96_legacy_project_-_rev,NaN
104660,1002226917.0,2406011712.0,2024-11-04,2024-10-25,DR,DO/2021-LP96,DO/2021-LP96REV,2024.0,550508.0,Rev-Domestic (Indian fund),...,-25.0,-25.0,-25.0,None,DO/2021-LP96,Class of '96 Legacy Project,DO/2021-LP96REV,Class of '96 Legacy Project - REV,class_of_'96_legacy_project_-_rev,NaN


In [32]:
lp_data.loc[lp_data['Project definition'] == 'DO/2021-LP96', 'Value in Obj. Crcy'].sum()

np.float64(-1943914.0)

In [15]:
final_data.head()

,Document Number,Ref. document number,Posting Date,Document Date,Document type,Project definition,Object,Fiscal Year,Cost Element,Cost element descr.,...,Vbl. value/TranCurr.,Vbl. value/Obj. curr,Var.val.in rep.cur.,Purchase order text,WBS,WBS Details,Sub WBS,SUB WBS Details,Project Id,Category
290,1002070478.0,2406001521.0,2024-05-06,2024-05-01,DR,DO/2009-HATS,DO/2009-HATSREV,2024.0,550508.0,Rev-Domestic (Indian fund),...,-501.00,-501.00,-501.00,None,DO/2009-HATS,HOSTEL ALUMNI TEAM STEWARDSHIP,DO/2009-HATSREV,Hostels - Receipt,hostels_-_receipt,Infrastructure
392,1002166308.0,2426005235.0,2024-08-29,2024-08-29,SB,DO/2009-INSF,DO/2009-INSFREV,2024.0,550509.0,Rev-General Expence,...,-51570.48,-51570.48,-51570.48,None,DO/2009-INSF,INFRASTRUCTURE ( STUDENT FACILITIES ),DO/2009-INSFREV,INFRASTRUCTURE ( STUDENT FACILITIES ),infrastructure_(_student_facilities_),NaN
413,1002161931.0,2426005015.0,2024-08-23,2024-08-23,SB,DO/2009-KRAS,DO/2009-KRAS001,2024.0,550509.0,Rev-General Expence,...,2475.00,2475.00,2475.00,None,DO/2009-KRAS,KERSAAP RUSI ANJIRBAG SCHOLARSHIP,DO/2009-KRAS001,KERSAAP RUSI ANJIRBAG SCHOLARSHIP,kersaap_rusi_anjirbag_scholarship,NaN
507,1002169039.0,2449006025.0,2024-09-03,2024-09-03,SI,DO/2009-MNGP,DO/2009-MNGP001,2024.0,510162.0,EXP-Awards,...,10000.00,10000.00,10000.00,None,DO/2009-MNGP,PROF.M.N.GOPALAN PRIZE,DO/2009-MNGP001,PROF.M.N.GOPALAN PRIZE,prof.m.n.gopalan_prize,NaN
517,1002167118.0,2426005310.0,2024-08-30,2024-08-30,SB,DO/2009-MNGP,DO/2009-MNGP001,2024.0,550054.0,REV-Awards,...,-10000.00,-10000.00,-10000.00,None,DO/2009-MNGP,PROF.M.N.GOPALAN PRIZE,DO/2009-MNGP001,PROF.M.N.GOPALAN PRIZE,prof.m.n.gopalan_prize,NaN


In [31]:
final_data.to_csv('Final_Data.csv', index=False, quoting=1, lineterminator='\r\n')